# Basic download and plotting with python
This notebook show how the data can be dowloaded/plotted from python

In [3]:
# general import, and set download path
import bokeh.io
bokeh.io.output_notebook()
# import geoviews as gv
# import geoviews.feature as gf
# gv.extension('bokeh','matplotlib')
import pandas as pd
import xarray as xr
# import rasterio as rio
# import rioxarray # geospatial extension for xarray
import os
import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
# import cartopy
# from tqdm.auto import tqdm


download_path="/tmp/cyclobs"
os.makedirs(download_path,exist_ok=True)

Loading BokehJS ...

In [4]:
# make the resquest using cyclobs api, and store the result in a pandas dataframe
request_url="https://cyclobs.ifremer.fr/app/api/getData?sid=al122017&instrument=C-Band_SAR&product_type=gridded&include_cols=all"
df_request = pd.read_csv(request_url)


# add download path
df_request['path'] = df_request['data_url'].map(lambda x : os.path.join(download_path,os.path.basename(x)))
df_request

,data_url,mission,mission_short,instrument,instrument_short,acquisition_start_time,bounding_box,dist_eye_centroid,eye_in_acq,sid,cyclone_name,cyclone_start_date,cyclone_stop_date,track_date,vmax (m/s),track_point,basin,maximum_cyclone_category,path
0,https://cyclobs.ifremer.fr/static/sarwing_data...,RADARSAT-2,RS2,C-Band Synthetic Aperture Radar,C-Band SAR,2017-09-06 21:15:19,"POLYGON ((-47.297268 5.730425, -42.684444 6.71...",81191.925946,True,al122017,JOSE,2017-09-03 18:00:00,2017-09-23 06:00:00,2017-09-06 21:15:00,35,POINT (-45.96666666666667 13.925),NaN,cat-4,/tmp/cyclobs/rs2--owi-cm-20170906t211519-20170...
1,https://cyclobs.ifremer.fr/static/sarwing_data...,SENTINEL-1 A,S1A,C-Band Synthetic Aperture Radar,C-Band SAR,2017-09-08 22:02:17,"POLYGON ((-58.796684 12.911622, -56.534523 13....",33237.219270,True,al122017,JOSE,2017-09-03 18:00:00,2017-09-23 06:00:00,2017-09-08 22:00:00,69,POINT (-58.46666666666666 16.6),NaN,cat-4,/tmp/cyclobs/s1a-iw-owi-cm-20170908t220217-201...


In [ ]:
# download 'data_url' to 'path' with wget, and read files
projection=ccrs.Mercator()
datasets = []
for idx,entry in tqdm(df_request.iterrows(),total=df_request.shape[0]):
    ret = os.system('cd %s ; wget -N  %s' % (os.path.dirname(entry['path']),entry['data_url']))
    if ret == 0 : 
        ds = xr.open_dataset(entry['path'])
        datasets.append( ds.rio.reproject(projection.proj4_params))
        
    else:
        datasets.append(None) # error fetching file
df_request['dataset'] = datasets


In [ ]:
df_request['dataset'].iloc[0]['wind_speed']

In [ ]:

gv_list=[gf.coastline.opts(projection=projection)]
for ds in df_request['dataset']:
    gv_list.append(gv.Image(ds['wind_speed'].squeeze()[::5,::5],crs=projection).opts(cmap='jet',tools=['hover']))
    
gv.Overlay(gv_list).options(width=800, height=500)



# Notebook installation

To run and customize this notebook on your computer, you will need to install some packages with conda:


```
conda create -n cyclobs python=3 cartopy

conda activate cyclobs

pip install xarray netcdf4 geoviews pandas rioxarray tqdm ipywidgets

wget https://gitlab.ifremer.fr/cyclobs/user_manual/-/raw/master/user_manual.ipynb
```

To start the notebook :

`jupyter notebook`

